In [1]:
import os
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import json
import pandas as pd

# Using Splinter

* Little googling guided me a little bit and it turns out the splinter is able to execute different languages and allows to read all the content. Simple requests does read only the html portion.

# 1. NASA Mars News

In [2]:
executable_path = {"executable_path": "C:/Users/aydin/Downloads/chromedriver_win32/chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(news_url)

In [5]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
#print(soup.prettify())

In [6]:
titles = []
para = []
for title in soup.find_all('div', class_='content_title'):
    if title.a:
        titles.append(title.a.text)
for paragraph in soup.find_all('div', class_='article_teaser_body'):
    if paragraph.text == False:
        para.append('False')
    else: 
        para.append(paragraph.text)
print(f"Number of titles: {len(titles)}\nNumber of Paragraphs: {len(para)}\n")

print(f"\033[1m{titles[0]}:\033[0m\n{para[0]}")

latest_title = titles[0]
latest_para = para[0]

Number of titles: 52
Number of Paragraphs: 40

NASA's Curiosity Mars Rover Snaps Its Highest-Resolution Panorama Yet:
To go along with the stunning 1.8-billion-pixel image, a new video offers a sweeping view of the Red Planet.


We can see that titles are greater in number than the paragraph texts. Since the assignment asks to fetch only the first title and associated text we just have to visually verify whether both elements are what we are looking for.  

# 2. JPL Mars Space Images - Featured Image

In [7]:
mars_pic_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(mars_pic_url)
mars_pic_html = browser.html
mars_pic_soup = BeautifulSoup(mars_pic_html, 'html.parser')

### Small code to fetch the largesize image

In [8]:

high_res_pics = []
image_url = ''
for anchors in mars_pic_soup.find_all('a', class_='fancybox'):
    if 'largesize' in anchors['data-fancybox-href']:
        image_url = 'jpl.nasa.gov' + anchors['data-fancybox-href']
        high_res_pics.append(image_url)
featured_image_url = high_res_pics[0]
print(featured_image_url)

jpl.nasa.gov/spaceimages/images/largesize/PIA23623_hires.jpg


# 3. Mars Weather - Scraping Twitter

* For some reason splinter did not work while trying to fetch the weather information from twitter. It kept delivering the Login and Sign in pages. So I needed to go back to requests the url.

In [9]:
url_twitter = "https://twitter.com/marswxreport?lang=en" 
twitter_html = requests.get(url_twitter).text
twitter_html = twitter_html.replace("\n", ", ")
twitter_soup = BeautifulSoup(twitter_html, 'html.parser')

### Small code for scraping Mars weather information

In [10]:
weather_list = []
for weather in twitter_soup.find_all('div', class_= "js-tweet-text-container"):
    if "InSight" in weather.p.text:
        weather_list.append(weather.p.getText().split('InSight')[1])

#latest Mars Weather stats
mars_weather = weather_list[0].split('pic.twitter.com')[0]
mars_weather

' sol 451 (2020-03-03) low -93.6ºC (-136.4ºF) high -10.3ºC (13.4ºF), winds from the SSE at 6.3 m/s (14.2 mph) gusting to 19.9 m/s (44.5 mph), pressure at 6.30 hPa'

# 4. Mars Facts - Web scraping with Pandas and BeautifulSoup

In [11]:
mars_facts_url = "https://space-facts.com/mars/" 
response = requests.get(mars_facts_url)
soup = BeautifulSoup(response.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
mars_html_table = df[0].to_html()
print(mars_html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Recorded